In [15]:
__author__ = "Yicheng Li"
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import random
from sklearn import preprocessing
import tensorflow as tf
import os

dir_path = ''
df = pd.read_pickle(dir_path+'df_hourly_poloniex.pickle')
df = df.dropna()

df.head()

,USDT_BTC_high,USDT_BTC_low,USDT_BTC_close,USDT_BTC_open,USDT_BTC_volume,USDT_BTC_quoteVolume,USDT_BTC_weighted_mean,USDT_BTC_pctChange,USDT_ETH_high,USDT_ETH_low,...,BTC_LTC_weighted_mean,BTC_LTC_pctChange,BTC_XRP_high,BTC_XRP_low,BTC_XRP_close,BTC_XRP_open,BTC_XRP_volume,BTC_XRP_quoteVolume,BTC_XRP_weighted_mean,BTC_XRP_pctChange
time,,,,,,,,,,,,,,,,,,,,,
2016-01-02 12:00:00,432.5000,432.50,432.500000,432.50000,40.041239,0.092581,432.500000,2.220446e-16,0.959136,0.959136,...,0.008063,-0.002293,0.000014,0.000014,0.000014,0.000014,0.033605,2408.822942,0.000014,-0.002859
2016-01-02 13:00:00,432.5000,432.50,432.500000,432.50000,0.000000,0.000000,432.986941,1.125876e-03,0.959136,0.959136,...,0.008060,-0.000333,0.000014,0.000014,0.000014,0.000014,0.000000,0.000000,0.000014,0.004704
2016-01-02 14:00:00,437.3635,432.48,433.336667,433.52799,359.269753,0.828819,433.473883,1.124610e-03,0.959136,0.957000,...,0.008073,0.001623,0.000014,0.000014,0.000014,0.000014,1.141981,81071.098773,0.000014,0.004682
2016-01-02 15:00:00,432.4800,432.48,432.480000,432.48000,60.859598,0.140722,432.480000,-2.292832e-03,0.957000,0.957000,...,0.008089,0.002002,0.000014,0.000014,0.000014,0.000014,2.120423,150622.792769,0.000014,-0.000492
2016-01-02 16:00:00,432.4800,432.48,432.480000,432.48000,0.000000,0.000000,432.350000,-3.005919e-04,0.957000,0.957000,...,0.008079,-0.001224,0.000014,0.000014,0.000014,0.000014,0.491516,35178.793196,0.000014,-0.007526


In [16]:
df2 = pd.read_pickle(dir_path+'df_hourly_news_sentiment_reach.pickle')
df2 = df2.reset_index()

df2['date'] = df2['datetime']
df2['datetime'] = pd.to_datetime(df2['datetime'],unit='ms')

df2 = df2.set_index('datetime')
df2 = df2.rename(columns={'date':'timestamp'})

In [19]:
df3 = pd.read_pickle(dir_path+'df_hourly_social_sentiment.pickle')

df3 = df3.reset_index()

df3['date'] = df3['datetime']
df3['datetime'] = pd.to_datetime(df3['datetime'],unit='ms')

df3 = df3.set_index('datetime')
df3 = df3.rename(columns={'date':'timestamp'}).drop(columns='timestamp')

,social_neutral,social_positive,social_negative,social_sentiment_mean,social_daily_sentiment_mean
datetime,,,,,
2017-02-02 00:00:00,11.0,13.0,1.0,0.424115,0.250033
2017-02-02 01:00:00,19.0,14.0,4.0,0.288782,0.250033
2017-02-02 02:00:00,12.0,7.0,2.0,0.205475,0.250033
2017-02-02 03:00:00,10.0,12.0,3.0,0.367691,0.250033
2017-02-02 04:00:00,16.0,10.0,3.0,0.233968,0.250033


In [20]:
df4 = df.join(df2).join(df3).dropna()

In [21]:
scaler_features = preprocessing.MinMaxScaler(feature_range=(0.1, 1))

In [22]:
# function to create train, validation, test data given sequence length
def load_data(df, seq_len, feature_set, test_size=-1):
    # prepare one-hot labels
    labels = df['USDT_BTC_pctChange'].as_matrix().reshape([-1,1])
    labels = np.concatenate([(labels > 3e-3)*1, ((3e-3 > labels)&(labels > -3e-3))*1, (labels < -3e-3)*1],1)
    
    data_raw = df.as_matrix() # convert to numpy array
    # fit scaler
    data_raw = scaler_features.fit_transform(data_raw[:, feature_set])
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len + 1): 
        data.append(data_raw[index: index + seq_len, :])
    
    data = np.array(data)
    
    if test_size == -1: # take train and valid sets first
        n_train_valid_pairs = 3
        each_train_set_size_pct = 25
        each_valid_set_size_pct = 5

        each_train_set_size = round(each_train_set_size_pct/100*data.shape[0])
        each_valid_set_size = round(each_valid_set_size_pct/100*data.shape[0])

        x_train_sets = []
        y_train_sets = []
        x_valid_sets = []
        y_valid_sets = []
        used = 0

        for i in range(n_train_valid_pairs):
            x_train_sets.append(data[used : used + each_train_set_size,:-1,:]) # cannot see last day, which we aim to predict
            y_train_sets.append(labels[used + seq_len-1 : used + each_train_set_size + seq_len-1, :])
            used += each_train_set_size

            x_valid_sets.append(data[used : used + each_valid_set_size,:-1,:])
            y_valid_sets.append(labels[used + seq_len-1 : used + each_valid_set_size + seq_len-1, :])
            used += each_valid_set_size

        x_test = data[used : , :-1, :]
        y_test = labels[seq_len-1 + used : , :]

        x_train = np.concatenate(x_train_sets, axis=0)
        y_train = np.concatenate(y_train_sets, axis=0)
        x_valid = np.concatenate(x_valid_sets, axis=0)
        y_valid = np.concatenate(y_valid_sets, axis=0)
    
    else: # take test set first
        labels = labels[seq_len-1:] # so labels and data has same length
        x_test = data[-test_size : , :-1, :]
        y_test = labels[-test_size : , :]
        
        valid_start = data.shape[0] - test_size - int(test_size/2)
        x_valid = data[valid_start:-test_size, :-1, :]
        y_valid = labels[valid_start:-test_size, :]
        
        x_train = data[:valid_start, :-1, :]
        y_train = labels[:valid_start, :]
        
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]
        

In [75]:
# create train, test data
seq_len = 10 # choose sequence length
feature_set = [x for x in range(df4.shape[-1])] #[0,1,2,3,4,6,7]
x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(df4, seq_len, feature_set, test_size=1996) # test_size = 1996
# y_train = y_train.reshape([-1,1])
# y_valid = y_valid.reshape([-1,1])
# y_test = y_test.reshape([-1,1])
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (6697, 9, 69)
y_train.shape =  (6697, 3)
x_valid.shape =  (998, 9, 69)
y_valid.shape =  (998, 3)
x_test.shape =  (1996, 9, 69)
y_test.shape =  (1996, 3)


/home/shared/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [24]:
def f1_score_single(y_true, y_pred):
    TP = y_true.dot(y_pred) # zero or one
    FP = np.sum(y_pred > y_true) # sum over all k classes, zero or one
    FN = np.sum(y_pred < y_true) # sum over all k classes, zero or one
    
    if TP == 0: return 0.
    p = 1. * TP / (TP + FP)
    r = 1. * TP / (TP + FN)
    return 2 * p * r / (p + r)
    
def acc(y_true, y_pred):
    return np.mean([f1_score_single(x, y) for x, y in zip(y_true, y_pred)])

from sklearn.metrics import f1_score

def F1(y_true, y_pred):
    y_true = np.argmax(y_true, axis=1)
    y_pred = np.argmax(y_pred, axis=1)
    return f1_score(y_true, y_pred, average='macro')  

## baseline

In [26]:
y_pred = np.roll(y_valid,1, axis=0)
print('baseline dev_acc=',acc(y_valid[1:], y_pred[1:]))
y_pred = np.roll(y_test,1, axis=0)
y_pred[0] = y_valid[-1] # be careful here
print('baseline test_acc=',acc(y_test, y_pred))

baseline dev_acc= 0.39618856569709127
baseline test_acc= 0.3712424849699399


In [65]:
index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

# function to get the next batch
def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size
        
    end = index_in_epoch
    return x_train[perm_array[start:end]], y_train[perm_array[start:end]]

x_1000_train, y_1000_train = get_next_batch(1000) # special batch of 1000 records in training set

In [104]:
class LSTM_Model(object):
    def __init__(self, seq_len, use_batch_normalization):
        # parameters
        self.n_steps = seq_len-1 
        self.n_inputs = x_train.shape[-1]
        self.n_neurons = 50  # cell.state_size
        self.n_bins = 3 # be careful if you want to change this
        self.n_layers = 2
        self.batch_size = 100
        self.n_epochs = 0 # 0 means to train indefinitely
        self.train_set_size = x_train.shape[0]
        self.test_set_size = x_test.shape[0]
        self.keep_prob = tf.placeholder(tf.float32, [])
        self.beta = tf.placeholder(tf.float32, []) # to control l2 regularization
        self.is_training = tf.placeholder(tf.bool)
        self.max_gradient_norm = 5
        with tf.variable_scope("LSTM_Model", initializer=tf.contrib.layers.xavier_initializer()):
            self.X = tf.placeholder(tf.float32, [None, self.n_steps, self.n_inputs])
            self.y = tf.placeholder(tf.float32, [None, self.n_bins])

            with tf.variable_scope("LSTM_set_1"):
                set1_layers = [tf.contrib.rnn.LSTMCell(num_units=self.n_neurons, \
                                                  initializer=tf.contrib.layers.xavier_initializer(), \
                                                  activation=tf.nn.elu)
                         for layer in range(self.n_layers)]

                set1_multi_layer_cell = tf.contrib.rnn.MultiRNNCell(set1_layers)

                outputs_1, _ = tf.nn.dynamic_rnn(set1_multi_layer_cell, self.X[:,:,:-13], dtype=tf.float32)
                
            with tf.variable_scope("LSTM_set_2"):
                set2_layers = [tf.contrib.rnn.LSTMCell(num_units=self.n_neurons, \
                                                  initializer=tf.contrib.layers.xavier_initializer(), \
                                                  activation=tf.nn.elu)]

                set2_multi_layer_cell = tf.contrib.rnn.MultiRNNCell(set2_layers)

                outputs_2, _ = tf.nn.dynamic_rnn(set2_multi_layer_cell, self.X[:,:,-13:], dtype=tf.float32)
            
            outputs_1 = outputs_1[:,-1,:] # take last timestep
            
            outputs_2 = outputs_2[:,-1,:]
            
            stacked_outputs = tf.concat([outputs_1, outputs_2], axis = -1)
            
#             stacked_outputs = tf.nn.dropout(stacked_outputs, self.keep_prob) # dropout after LSTM
            # 'outputs' is a tensor of shape [batch_size, n_steps, n_neurons(cell.state_size)]
            
            stacked_outputs = tf.layers.dense(stacked_outputs, 20, activation=tf.nn.elu)  # first FC
            # batch normalization
            if use_batch_normalization:
                stacked_outputs = tf.contrib.layers.batch_norm(stacked_outputs, center=True, \
                                                               scale=False, is_training=self.is_training, scope='bn')
            stacked_outputs = tf.nn.dropout(stacked_outputs, self.keep_prob)
            stacked_outputs = tf.layers.dense(stacked_outputs, self.n_bins, activation=tf.nn.elu)  # second FC
            
            self.final_logits = tf.reshape(stacked_outputs, [-1, self.n_bins])
            
            self.final_logits = tf.nn.dropout(self.final_logits, self.keep_prob) # dropout
            
            self.indices = tf.argmax(self.final_logits, axis=-1) # (batchsize, 1)
            self.preds = tf.one_hot(self.indices, depth=self.n_bins)
            
            self.each_loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.final_logits, labels=self.y)
            self.loss = tf.reduce_mean(self.each_loss) 
           
            params = tf.trainable_variables()
            
            self.loss += self.beta * tf.add_n([ tf.nn.l2_loss(v) for v in params ])
            
            gradients = tf.gradients(self.loss, params)
            self.gradient_norm = tf.global_norm(gradients)
            clipped_gradients, _ = tf.clip_by_global_norm(gradients, self.max_gradient_norm)
            clipped_norm = tf.global_norm(clipped_gradients)
            self.param_norm = tf.global_norm(params)
            self.learning_rate_placeholder = tf.placeholder(tf.float32, [], name='learning_rate')
            optimizer = tf.train.RMSPropOptimizer(learning_rate=self.learning_rate_placeholder) 
            # training_op = optimizer.minimize(loss)
            self.training_op = optimizer.apply_gradients(zip(clipped_gradients, params))

            # initialize parameters
#             sess = tf.Session()
            self.global_step = tf.Variable(0, name='global_step', trainable=False)
            self.saver = tf.train.Saver(max_to_keep=2)
            self.bestmodel_saver = tf.train.Saver(max_to_keep=2)
    
    def train(self, session, experiment_name, keep_prob_val, beta):
        
        bestmodel_dir = experiment_name+'/best_ckpt'
        bestmodel_ckpt_path = bestmodel_dir+'/best.ckpt'
        best_valid_f1 = None
        # Make bestmodel dir if necessary
        if not os.path.exists(bestmodel_dir):
            os.makedirs(bestmodel_dir)
        
        ckpt = tf.train.get_checkpoint_state(experiment_name)
        v2_path = ckpt.model_checkpoint_path + ".index" if ckpt else ""
        if ckpt and (tf.gfile.Exists(ckpt.model_checkpoint_path) or tf.gfile.Exists(v2_path)):
            self.saver.restore(sess, ckpt.model_checkpoint_path)
            iteration = self.global_step.eval(session=sess) # get last global_step
            print("Start from iteration:", iteration)
            lr = 1e-3
        else:
            print('There is not saved parameters. Creating model with fresh parameters.')
            sess.run(tf.global_variables_initializer())
            iteration = 0
            lr = 1e-2 # should fix this...

        old_loss = 1000
        
        while self.n_epochs == 0 or iteration*self.batch_size/self.train_set_size < self.n_epochs:
            iteration = iteration + 1
            x_batch, y_batch = get_next_batch(self.batch_size) # fetch the next training batch 

            # train on this batch
            sess.run(self.training_op, feed_dict={self.X: x_batch, self.y: y_batch, self.learning_rate_placeholder:lr, \
                                                 self.keep_prob: keep_prob_val, self.beta: beta, self.is_training: True})

            if iteration % 100 == 0:
                y_1000_train_pred = []
                loss_val = []
                for idx in range(0, 1000, 100):
                    y_1000_train_pred_chunk, loss_val_chunk, param_norm_val, grad_norm_val = \
                        sess.run([self.preds, self.loss, self.param_norm, self.gradient_norm],\
                            feed_dict={self.X: x_1000_train[idx:idx+100,:,:], self.y:y_1000_train[idx:idx+100,:], \
                                        self.learning_rate_placeholder:lr, self.keep_prob: 1, \
                                        self.beta: beta, self.is_training: False})
                    y_1000_train_pred.append(y_1000_train_pred_chunk)
                    loss_val.append(loss_val_chunk)
                y_1000_train_pred = np.concatenate(y_1000_train_pred, axis=0)
                loss_val = np.mean(loss_val)
                    
                if loss_val > old_loss * 1.2:
                    lr /= 2
                old_loss = loss_val

                y_valid_pred = sess.run(self.preds, feed_dict={self.X: x_valid, self.keep_prob: 1,\
                                                              self.is_training: False})
                
                valid_f1 = acc(y_valid, y_valid_pred)
                print('%.2f epochs, iter %d: train_loss = %.9f, param_norm = %.3f, grad_norm = %.3f, train_acc/valid_acc = %.6f/%.6f' \
                      %(iteration*self.batch_size/self.train_set_size, iteration, loss_val, param_norm_val, grad_norm_val, \
                        acc(y_1000_train, y_1000_train_pred), \
                        valid_f1))

                if best_valid_f1 is None or valid_f1 > best_valid_f1:
                    best_valid_f1 = valid_f1
                    self.global_step.assign(iteration).eval(session=sess)
                    print("======New best valid F1. Saving to %s..." % bestmodel_ckpt_path)
                    self.bestmodel_saver.save(sess, bestmodel_ckpt_path, global_step=self.global_step)
                
            if iteration % 1000 == 0:
                self.global_step.assign(iteration).eval(session=sess) # set and update(eval) global_step with index, i
                save_path = self.saver.save(sess, "./"+experiment_name+"/model.ckpt", global_step=self.global_step)
                print('Saved parameters to %s' % save_path)

## run experiment

In [96]:
experiment_name="04-29-03"

In [128]:
tf.reset_default_graph()
lstm_model = LSTM_Model(seq_len=seq_len, use_batch_normalization=False)
sess = tf.Session()

In [102]:
# sanity check
sess.run(tf.global_variables_initializer())
# first check random loss
sanity_loss = sess.run(lstm_model.loss, feed_dict={lstm_model.X: x_1000_train, lstm_model.y:y_1000_train, \
                                        lstm_model.learning_rate_placeholder:1e-3, lstm_model.keep_prob: 1, \
                                                   lstm_model.beta: 0, lstm_model.is_training: False})
print(sanity_loss) # should be around 1.09

sanity_loss = sess.run(lstm_model.loss, feed_dict={lstm_model.X: x_1000_train, lstm_model.y:y_1000_train, \
                                        lstm_model.learning_rate_placeholder:1e-3, lstm_model.keep_prob: 1, \
                                                   lstm_model.beta: 0.005, lstm_model.is_training: False})
print(sanity_loss) # should be around 1.09

x_100_train, y_100_train = get_next_batch(100)
iter_ = 0
while True:
    iter_ += 1
    _, y_100_train_pred, loss_val = sess.run([lstm_model.training_op, lstm_model.preds, lstm_model.loss], \
                                             feed_dict={lstm_model.X: x_100_train, lstm_model.y:y_100_train, \
                                        lstm_model.learning_rate_placeholder:1e-3, lstm_model.keep_prob: 1, \
                                                        lstm_model.beta: 0, lstm_model.is_training: True})
    if iter_ % 50 == 0:
        print('iter %d: train_loss = %.9f, train_F1= %.6f' \
                      %(iter_, loss_val, F1(y_100_train, y_100_train_pred)))
    if loss_val < 0.01:
        break

1.1071486
2.8860598


/home/shared/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


iter 50: train_loss = 1.083558083, train_F1= 0.193853
iter 100: train_loss = 1.057697654, train_F1= 0.352005
iter 150: train_loss = 1.014568448, train_F1= 0.414587
iter 200: train_loss = 0.943552732, train_F1= 0.477160
iter 250: train_loss = 0.872360945, train_F1= 0.539590
iter 300: train_loss = 0.759383440, train_F1= 0.651793
iter 350: train_loss = 0.748818457, train_F1= 0.582966
iter 400: train_loss = 0.633967996, train_F1= 0.739263
iter 450: train_loss = 0.569624782, train_F1= 0.797337
iter 500: train_loss = 0.599731505, train_F1= 0.690629
iter 550: train_loss = 0.474582285, train_F1= 0.810825
iter 600: train_loss = 0.316559494, train_F1= 0.822157
iter 650: train_loss = 0.547431171, train_F1= 0.817095
iter 700: train_loss = 0.217297688, train_F1= 0.907511
iter 750: train_loss = 0.203553900, train_F1= 0.926099
iter 800: train_loss = 0.221336916, train_F1= 0.927721
iter 850: train_loss = 0.164127231, train_F1= 0.947208
iter 900: train_loss = 0.201678574, train_F1= 0.908490
iter 950: t

In [ ]:
lstm_model.train(session=sess, experiment_name=experiment_name, keep_prob_val=1, beta=1e-4)
# IMPORTANT:
# when you think F1 is not going to improve anymore, wait another 50-100 epochs. 
# if you see any better iteration that has not appeared before, keep waiting.

There is not saved parameters. Creating model with fresh parameters.
1.49 epochs, iter 100: train_loss = 1.112728834, param_norm = 20.052, grad_norm = 0.538, train_acc/valid_acc = 0.387000/0.236473
======New best valid F1. Saving to 04-29-03/best_ckpt/best.ckpt...
2.99 epochs, iter 200: train_loss = 1.109373689, param_norm = 14.710, grad_norm = 0.002, train_acc/valid_acc = 0.387000/0.397796
======New best valid F1. Saving to 04-29-03/best_ckpt/best.ckpt...
4.48 epochs, iter 300: train_loss = 1.083248496, param_norm = 9.477, grad_norm = 0.130, train_acc/valid_acc = 0.394000/0.413828
======New best valid F1. Saving to 04-29-03/best_ckpt/best.ckpt...
5.97 epochs, iter 400: train_loss = 1.097630262, param_norm = 8.210, grad_norm = 0.627, train_acc/valid_acc = 0.366000/0.419840
======New best valid F1. Saving to 04-29-03/best_ckpt/best.ckpt...
7.47 epochs, iter 500: train_loss = 1.099191189, param_norm = 7.206, grad_norm = 0.208, train_acc/valid_acc = 0.373000/0.395792
8.96 epochs, iter 600

89.59 epochs, iter 6000: train_loss = 1.048379898, param_norm = 11.596, grad_norm = 0.279, train_acc/valid_acc = 0.455000/0.237475
Saved parameters to ./04-29-03/model.ckpt-6000
91.09 epochs, iter 6100: train_loss = 1.044320226, param_norm = 11.827, grad_norm = 0.344, train_acc/valid_acc = 0.451000/0.233467
92.58 epochs, iter 6200: train_loss = 1.045494080, param_norm = 11.714, grad_norm = 0.344, train_acc/valid_acc = 0.461000/0.233467
94.07 epochs, iter 6300: train_loss = 1.056118011, param_norm = 12.539, grad_norm = 0.299, train_acc/valid_acc = 0.445000/0.244489
95.57 epochs, iter 6400: train_loss = 1.045501828, param_norm = 11.740, grad_norm = 0.372, train_acc/valid_acc = 0.451000/0.235471
97.06 epochs, iter 6500: train_loss = 1.051348567, param_norm = 12.093, grad_norm = 0.301, train_acc/valid_acc = 0.438000/0.258517
98.55 epochs, iter 6600: train_loss = 1.055734277, param_norm = 11.980, grad_norm = 0.427, train_acc/valid_acc = 0.432000/0.236473
100.04 epochs, iter 6700: train_loss

179.18 epochs, iter 12000: train_loss = 1.041124582, param_norm = 11.490, grad_norm = 0.381, train_acc/valid_acc = 0.463000/0.227455
Saved parameters to ./04-29-03/model.ckpt-12000
180.68 epochs, iter 12100: train_loss = 1.062494159, param_norm = 11.503, grad_norm = 0.309, train_acc/valid_acc = 0.424000/0.244489
182.17 epochs, iter 12200: train_loss = 1.058387518, param_norm = 11.509, grad_norm = 0.456, train_acc/valid_acc = 0.420000/0.251503
183.66 epochs, iter 12300: train_loss = 1.039213538, param_norm = 11.576, grad_norm = 0.389, train_acc/valid_acc = 0.470000/0.235471
185.16 epochs, iter 12400: train_loss = 1.042135477, param_norm = 11.680, grad_norm = 0.590, train_acc/valid_acc = 0.459000/0.226453
186.65 epochs, iter 12500: train_loss = 1.042138100, param_norm = 11.558, grad_norm = 0.367, train_acc/valid_acc = 0.462000/0.246493
188.14 epochs, iter 12600: train_loss = 1.040363669, param_norm = 11.679, grad_norm = 0.527, train_acc/valid_acc = 0.465000/0.222445
189.64 epochs, iter 1

In [130]:
# load best checkpoint (based on dev f1) and evaluate
ckpt = tf.train.get_checkpoint_state(experiment_name+'/best_ckpt')
v2_path = ckpt.model_checkpoint_path + ".index" if ckpt else ""
if ckpt and (tf.gfile.Exists(ckpt.model_checkpoint_path) or tf.gfile.Exists(v2_path)):
    lstm_model.saver.restore(sess, ckpt.model_checkpoint_path)
else:
    raise ValueError('What? you dont have a best checkpoint?')

y_1000_train_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_1000_train, lstm_model.keep_prob: 1, \
                                                          lstm_model.beta:0, lstm_model.is_training: False})
print("train acc:",acc(y_1000_train, y_1000_train_pred))
y_valid_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_valid, lstm_model.keep_prob: 1, \
                                                     lstm_model.beta:0, lstm_model.is_training: False})
print("dev acc:",acc(y_valid, y_valid_pred))
y_test_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_test, lstm_model.keep_prob: 1, \
                                                    lstm_model.beta:0, lstm_model.is_training: False})
print("test acc:",acc(y_test, y_test_pred))

INFO:tensorflow:Restoring parameters from 04-29-03/best_ckpt/best.ckpt-87500
train acc: 0.602
dev acc: 0.46993987975951906
test acc: 0.40180360721442887
